In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__valor = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getValor(self):
        return self.__valor
    
    def setValor(self, valor):
        self.__valor = valor
        
    def getArestas(self):
        return self.__arestas

    def removerAresta(self, aresta):
        self.__arestas.remove(aresta)
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())
    
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes

In [3]:
class Aresta:
    def __init__(self, vOrigem, vDestino, peso, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__peso

In [4]:
from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getVerticeByValor(self, valor):
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None
    
    def getArestas(self):
        return self.__arestas
    
    def adicionarVertice(self, valor):
        if self.buscarPorValor(valor) != valor:# valor nao esta no grafo
            self.__vertices.add(Vertice(valor))
            return True
        return False
    
    def adicionarAresta(self, origem, destino, peso = 1, direcionada = True):
        try:
            verticeOrigem = self.getVerticeByValor(origem)
            verticeDestino = self.getVerticeByValor(destino)
            if (verticeOrigem or verticeDestino) is None:  # existem os vertices de origem e destino?
                print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
            self.__arestas.add(Aresta(verticeOrigem, verticeDestino, peso, direcionada))
        except AttributeError as error:
            print("Nao ha no grafo, vertices de origem ou de destino com os valores informados.")
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):
        if v not in visitados: # se v nao foi visitado
            visitados.append(v) # marca vertice como visitado
        if len(v.getAdjacentes(v)) == 0: # vertice escolhido nao tem adjacentes
            self.dfs(graph, next(iter(graph.getVertices())), visitados) # chamada recursiva pegando o proximo vertice do set
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(v): #percorre todos os adjacentes a ele
                if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                    self.dfs(graph, adjacente, visitados) # chamada recursiva para cada adjacente
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):
        fila.append(v)  # adiciona o vertice v a fila
        if v not in visitados:  # se vertice v nao esta em visitados
            visitados.append(v)  # adiciona vertice v a visitados
        while fila:  # enquanto houver vertices na fila
            vertice = fila.popleft()  # tira vertice ja visitado da fila
            if len(vertice.getArestasSaida()) == 0: # vertice escolhido nao tem adjacentes
                self.bfs(next(iter(self.getVertices())), visitados, fila) # chamada recursiva pegando o proximo vertice do set   
            else:
                for adjacente in vertice.getAdjacentes(v): #percorre todos os adjacentes a ele
                    if adjacente not in visitados: # se um dos adjacentes nao foi visitado
                        visitados.append(adjacente)  # insere o adjacente em visitados
                        self.bfs(adjacente, visitados, fila) # chamada recursiva pegando o proximo vertice do set
        return visitados  # retorna a lista de visitados  
    
    def buscarPorValor(self, valor):
        for v in self.bfs(next(iter(self.getVertices())), visitados = [], fila = deque([])):
            if valor == v.getValor():
                return valor
        return None
    
    def existeCaminhoEuler(self):
        pares = True
        impares = []
        for v in self.getVertices():
            if v.getGrau() % 2 != 0:
                pares = False
                impares.append(v)
        return pares or len(impares) == 2

    def removerAresta(self, peso, valorVOrigem, valorVDestino):
        arestaDeletar = None
        # Encontrando a aresta alvo pela busca de profundidade
        for v in self.bfs(next(iter(self.getVertices()))):
            if arestaDeletar != None:
                break
            arestas = v.getArestas()
            for a in arestas:
                if a.getValor() == peso:
                    arestaDeletar = a
                    break
        vOrigem = self.getVerticeByValor(valorVOrigem)
        vDestino = self.getVerticeByValor(valorVDestino)
        if arestaDeletar.getvOrigem() != vOrigem:
            print("Valor do vértice de origem é diferente do procurado.")
            return False
        if arestaDeletar.getvDestino() != vDestino:
            print("Valor do vértice de destino é diferente do procurado.")
            return False
        # Remove aresta da lista de arestas do Grafo
        self.getArestas().remove(arestaDeletar)
        # Remove aresta do vértice origem
        vOrigem.removerAresta(arestaDeletar)
        # Remove aresta do vértice destino
        vDestino.removerAresta(arestaDeletar)
        return True

    def removerVertice(self, valor):
        verticeRemover = self.getVerticeByValor(valor)
        if verticeRemover == None:
            print("Vértice com valor " + str(valor) + " não foi encontrado.")
            return False
        arestasRemover = verticeRemover.getArestas()
        for aresta in arestasRemover.copy():
            valorVOrigem = aresta.getvOrigem().getValor()
            valorVDestino = aresta.getvDestino().getValor()
            self.removerAresta(aresta.getValor(), valorVOrigem, valorVDestino)
        self.getVertices().remove(verticeRemover)
        return True

In [5]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )

In [6]:
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [7]:
G.adicionarVertice(20)
G.adicionarAresta(5, 15, 10, True)

Nao ha no grafo, vertices de origem ou de destino com os valores informados.


In [8]:
for v in G.getVertices():
    print(v.getValor(), end="\t")

4	20	1	5	2	3	

In [9]:
v1.getGrau()

2

In [10]:
G.checkHandShakingLemma()

False

#### Testando busca em profundidade

In [11]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
# for vertice in G.getVertices():
#     print(f"Busca em profundidade, iniciando com o vértice {vertice.getValor()}:")
#     for v in G.dfs(G, vertice, visitados=[]):
#         print(str(v.getValor())+"\t", end="")
#     print("\n.................................................")  

In [12]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
# for vertice in G.getVertices():
#     print(f"Busca em largura, iniciando com o vértice {vertice.getValor()}:")
#     for v in  G.bfs(vertice, visitados = [], fila = deque([])):
#         print(str(v.getValor())+"\t", end="")
#     print("\n............................................")   

In [13]:
# valor = int(input("Valor procurado: "))
# if G.buscarPorValor(valor) == None:
#     print(f"{valor} não encontrado no grafo")
# else:
#     print(f"{valor} encontrado no grafo")
# [print(f"{v.getValor()} encontrado no grafo.") for v in  G.bfs(v1, visitados = [], fila = deque([])) if n == v.getValor() ]

In [14]:
if G.existeCaminhoEuler():
    print("Grafo contém um caminho Euleriano")
else:
    print("Grafo NÃO contém um caminho Euleriano")

Grafo contém um caminho Euleriano


In [15]:
for a in G.getArestas():
    print(a.getvOrigem().getValor(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getValor(), end="\t")

1 ---> 2	2 ---> 3	4 ---> 6	4 ---> 1	4 ---> 5	3 ---> 4	

In [16]:
G.removerAresta(10, 1, 2)

True

In [17]:
G.removerVertice(6)

True

In [18]:
for a in G.getArestas():
    print(a.getvOrigem().getValor(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getValor(), end="\t")

2 ---> 3	4 ---> 1	4 ---> 5	3 ---> 4	

In [19]:
# Vértices antes
for v in G.getVertices():
    print(v.getValor(), end="\t")

2	4	5	3	1	

In [20]:
G.removerVertice(1)

True

In [21]:
# Vértices depois
for v in G.getVertices():
    print(v.getValor(), end="\t")

2	4	5	3	